# Look at accelerometer data 

Finding Zero velocity times by rail axis acceleration noise levels, making summary statistics for the noise levels across the whole day files.  Spot check graphs to see what works

In [1]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp

#Custome graph format style sheet
#plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = False
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.png'

#Standard cycle to make black and white images and dashed and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '-', '-', '-']))
plt.rc('axes', prop_cycle=default_cycler)
my_cmap = plt.get_cmap('gray')

In [2]:
#Extra Headers:
import os as os
import pywt as py
import statistics as st
import os as os
import random
import multiprocessing
from joblib import Parallel, delayed
import platform

from time import time as ti

In [3]:
import CoreFunctions as cf
from skimage.restoration import denoise_wavelet

## Choosing Platform
Working is beinging conducted on several computers, and author needs to be able to run code on all without rewriting..  This segment of determines which computer is being used, and sets the directories accordingly.

In [4]:
HostName = platform.node()

if HostName == "Server":
    Computer = "Desktop"   
elif HostName[-6:] == 'wm.edu':
    Computer = "SciClone"
elif HostName == "SchoolLaptop":
    Computer = "LinLap"
elif HostName == "WTC-TAB-512":
    Computer = "PortLap"
else:
    Computer = "WinLap"

In [5]:
if Computer == "SciClone":
    location = '/sciclone/home20/dchendrickson01/image/'
elif Computer == "WinLap":
    location = 'C:\\Data\\'
elif Computer == "Desktop":
    location = "E:\\Backups\\Dan\\CraneData\\"
elif Computer == "LinLap":
    location = '/home/dan/Output/'
elif Computer == 'PortLap':
    location = 'C:\\users\\dhendrickson\\Desktop\\AccelData\\'

In [6]:
if Computer ==  "SciClone":
    rootfolder = '/sciclone/home20/dchendrickson01/'
    folder = '/sciclone/scr10/dchendrickson01/Recordings2/'
    imageFolder = '/sciclone/scr10/dchendrickson01/Move3Dprint/'
elif Computer == "Desktop":
    rootfolder = location
    folder = rootfolder + "Recordings2\\"
elif Computer =="WinLap":
    rootfolder = location
    folder = rootfolder + "Recordings2\\"   
elif Computer == "LinLap":
    rootfolder = '/home/dan/Data/'
    folder = rootfolder + 'Recordings2/'
elif Computer =='PortLap':
    rootfolder = location 
    folder = rootfolder + 'Recordings2\\'

## Global Variables

In [7]:
Saving = False
location = folder
Titles = True
Ledgends = True

f = 0


In [8]:
files = ['230418 recording1.csv','230419 recording1.csv','230420 recording1.csv','230421 recording1.csv',
         '230418 recording2.csv','230419 recording2.csv','230420 recording2.csv','230421 recording2.csv']

In [9]:
BeforeTamping = ['221206 recording1.csv','221207 recording1.csv','221208 recording1.csv','221209 recording1.csv',
         '221206 recording2.csv','221207 recording2.csv','221208 recording2.csv','221209 recording2.csv']


## Project Specific Functions

In [10]:
#Smooth = cf.Smoothing(ODataSet[:,3],2) #,50)
def SmoothMoves(file):
    #    if file[-3:] =='csv':
    ODataSet = np.genfromtxt(open(folder+file,'r'), delimiter=',',skip_header=0,missing_values=0,invalid_raise=False)
    SmoothX = denoise_wavelet(ODataSet[:,3], method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
    SmoothY = denoise_wavelet(ODataSet[:,4], method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
    SmoothZ = denoise_wavelet(ODataSet[:,5], method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
    SmoothX -= np.average(SmoothX)
    SmoothY -= np.average(SmoothY)
    SmoothZ -= np.average(SmoothZ)
    MoveMatrix = np.matrix([SmoothX, SmoothY, SmoothZ])
    return MoveMatrix
    #else:
    #    pass


## Process Files

In [11]:
LoopFiles = 3
loops = int(len(files) / LoopFiles) 
if len(files)%LoopFiles != 0:
    loops += 1


In [12]:
files


['230418 recording1.csv',
 '230419 recording1.csv',
 '230420 recording1.csv',
 '230421 recording1.csv',
 '230418 recording2.csv',
 '230419 recording2.csv',
 '230420 recording2.csv',
 '230421 recording2.csv']

In [13]:

st = ti()

Moves = []

for k in range(loops):
    if k == loops -1:
        tfiles = files[k*LoopFiles:]
    else:
        tfiles = files[k*LoopFiles:(k+1)*LoopFiles]
    #Results = Parallel(n_jobs=LoopFiles)(delayed(DeviationVelocity)(file) for file in tfiles)
    Results = Parallel(n_jobs=LoopFiles)(delayed(SmoothMoves)(file) for file in tfiles)
    #Results =[]
    #for file in tfiles:
    #    Results.append(DeviationVelocity(file))
    #    print(file, (ti()-st)/60.0)
    for result in Results:
        Moves.append(result)
    print(k, (ti()-st)/60.0)
        


0 1.6526994466781617
1 3.236815166473389
2 4.3306843638420105


## Try LSTM Stuff

https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

In [14]:
TimeSteps = 250
StepSize = 75
Features = np.shape(Moves[0])[0]

In [15]:
len(Moves)

8

In [16]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps, s_step = 1):
    X, y = list(), list()
    Steps_to_take = int(len(sequences) / s_step)
    for j in range(Steps_to_take):
        i = j * s_step
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [17]:
Sequences = []
Outputs = []
for move in Moves:
    Seq, Out = split_sequences(move.T,TimeSteps,StepSize)
    Sequences.append(Seq)
    Outputs.append(Out)
    

In [18]:
MoveSegments = []
for seq in Sequences:
    for mv in seq:
        MoveSegments.append(mv)
NextDataPoint = []
for out in Outputs:
    for pt in out:
        NextDataPoint.append(np.reshape(pt,(1,3)))

In [19]:
len(MoveSegments)

1713931

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Masking
from keras.models import Sequential
import tensorflow as tf


class LSTM_Autoencoder:
  def __init__(self, optimizer='adam', loss='mse'):
    self.optimizer = optimizer
    self.loss = loss
    self.n_features = Features
    self.timesteps = TimeSteps
    
  def build_model(self):
    timesteps = self.timesteps
    n_features = self.n_features
    model = Sequential()
    
    # Padding
    #model.add(Masking(mask_value=0.0, input_shape=(timesteps, n_features)))

    # Encoder
    model.add(LSTM(timesteps, activation='relu', input_shape=(TimeSteps, Features), return_sequences=True))
    model.add(LSTM(35, activation='relu', return_sequences=True))
    model.add(LSTM(6, activation='relu'))
    model.add(RepeatVector(timesteps))
    
    # Decoder
    model.add(LSTM(timesteps, activation='relu', return_sequences=True))
    model.add(LSTM(35, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(n_features)))
    
    model.compile(optimizer=self.optimizer, loss=self.loss, metrics=['accuracy'])
    model.summary()
    self.model = model
    
  def simple_model(self):
    
    # define model
    model = Sequential()
    model.add(LSTM(150, input_shape=(self.timesteps, self.n_features)))
    model.add(RepeatVector(n_timesteps_in))
    model.add(LSTM(150, return_sequences=True))
    model.add(TimeDistributed(Dense(n_features, activation='softmax')))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    self.model = model
    
  def fit(self, X, epochs=3, batch_size=32):
    #self.timesteps = np.shape(X)[0]
    self.build_model()
    
    #input_X = np.expand_dims(X, axis=1)
    self.model.fit(X, X, epochs=epochs, batch_size=batch_size)
    
  def predict(self, X):
    #input_X = np.expand_dims(X, axis=1)
    output_X = self.model.predict(X)
    reconstruction = np.squeeze(output_X)
    return np.linalg.norm(X - reconstruction, axis=-1)
  
  def plot(self, scores, timeseries, threshold=0.95):
    sorted_scores = sorted(scores)
    threshold_score = sorted_scores[round(len(scores) * threshold)]
    
    plt.title("Reconstruction Error")
    plt.plot(scores)
    plt.plot([threshold_score]*len(scores), c='r')
    plt.show()
    
    anomalous = np.where(scores > threshold_score)
    normal = np.where(scores <= threshold_score)
    
    plt.title("Anomalies")
    plt.scatter(normal, timeseries[normal][:,-1], s=3)
    plt.scatter(anomalous, timeseries[anomalous][:,-1], s=5, c='r')
    plt.show()


2023-07-26 09:53:00.219319: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
#split = int(len(Moves)*.9)
#Train_data = Moves[:split]
#Test_data = Moves[split:]
#Train_data = tf.ragged.constant(Train_data)
#Test_data = tf.ragged.constant(Test_data)

In [23]:
lstm_autoencoder2 = LSTM_Autoencoder(optimizer='adam', loss='msle')

In [24]:
lstm_autoencoder2.build_model()

2023-07-26 09:53:03.873613: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 09:53:03.883750: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 250, 250)          254000    
                                                                 
 lstm_1 (LSTM)               (None, 250, 35)           40040     
                                                                 
 lstm_2 (LSTM)               (None, 6)                 1008      
                                                                 
 repeat_vector (RepeatVector  (None, 250, 6)           0         
 )                                                               
                                                                 
 lstm_3 (LSTM)               (None, 250, 250)          257000    
                                                                 
 lstm_4 (LSTM)               (None, 250, 35)           40040     
                                                        

In [25]:
Batches = 32
NumbBatches = 1000

SamplesPerSet = Batches * NumbBatches


In [26]:
SamplesPerSet

3200

In [27]:
SetsNeeded = int(len(MoveSegments) / SamplesPerSet)
if  int(len(MoveSegments) / SamplesPerSet) != 0:
    SetsNeeded += 1
print(len(MoveSegments), SetsNeeded)

1713931 536


In [28]:
PercentPerSet = 1.0 / float(SetsNeeded)

PercentHoldOutForNext=1.0

In [ ]:
st = ti()

for i in range(SetsNeeded-1):
    PercentHoldOutForNext = 1.0 - (SamplesPerSet / len(MoveSegments))
    seq_train, seq_test, out_train, out_test = train_test_split(MoveSegments, NextDataPoint, test_size=PercentHoldOutForNext, shuffle=True, random_state=0)
    seq_train = np.asarray(seq_train)
    lstm_autoencoder2.model.fit(seq_train, seq_train, epochs=2, batch_size=32, verbose=1)
    MoveSegments = seq_test
    NextDataPoint = out_test
    print(str(i+1)+' of ' + str(SetsNeeded), (ti()-st)/60, ((ti()-st)/(i+1) * ( SetsNeeded -1) - (ti()-st))/60/60)
    
    

Epoch 1/2
100/100 [==============================] - 178s 2s/step - loss: 4.2503e-05 - accuracy: 0.3446
Epoch 2/2
100/100 [==============================] - 150s 2s/step - loss: 4.2431e-05 - accuracy: 0.3450
1 of 536 5.498353819052379 48.935349625812634
Epoch 1/2
100/100 [==============================] - 136s 1s/step - loss: 4.7819e-05 - accuracy: 0.3391
Epoch 2/2
100/100 [==============================] - 142s 1s/step - loss: 4.7777e-05 - accuracy: 0.3385
2 of 536 10.159175670146942 45.12367219924927
Epoch 1/2
100/100 [==============================] - 150s 2s/step - loss: 4.7601e-05 - accuracy: 0.3461
Epoch 2/2
100/100 [==============================] - 148s 1s/step - loss: 4.7511e-05 - accuracy: 0.3429
3 of 536 15.158909130096436 44.80299834171931
Epoch 1/2
100/100 [==============================] - 130s 1s/step - loss: 4.6357e-05 - accuracy: 0.3428
Epoch 2/2
100/100 [==============================] - 163s 2s/step - loss: 4.5301e-05 - accuracy: 0.3459
4 of 536 20.06973661184311 44.

100/100 [==============================] - 186s 2s/step - loss: 5.1160e-05 - accuracy: 0.3554
Epoch 2/2
100/100 [==============================] - 141s 1s/step - loss: 5.1160e-05 - accuracy: 0.3554
33 of 536 170.94991360902787 43.341846800564916
Epoch 1/2
100/100 [==============================] - 158s 2s/step - loss: 4.9452e-05 - accuracy: 0.3560
Epoch 2/2
100/100 [==============================] - 152s 2s/step - loss: 4.9452e-05 - accuracy: 0.3560
34 of 536 176.1261117021243 43.2545009784644
Epoch 1/2
100/100 [==============================] - 178s 2s/step - loss: 5.3650e-05 - accuracy: 0.3555
Epoch 2/2
100/100 [==============================] - 142s 1s/step - loss: 5.3650e-05 - accuracy: 0.3555
35 of 536 181.48930064837137 43.211738262526566
Epoch 1/2
100/100 [==============================] - 171s 2s/step - loss: 5.1516e-05 - accuracy: 0.3545
Epoch 2/2
100/100 [==============================] - 139s 1s/step - loss: 5.1516e-05 - accuracy: 0.3545
36 of 536 186.6713469306628 43.124538

100/100 [==============================] - 133s 1s/step - loss: 5.3463e-05 - accuracy: 0.3503
Epoch 2/2
100/100 [==============================] - 138s 1s/step - loss: 5.3463e-05 - accuracy: 0.3503
65 of 536 327.9623473683993 39.52366750824146
Epoch 1/2
100/100 [==============================] - 143s 1s/step - loss: 5.2736e-05 - accuracy: 0.3546
Epoch 2/2
100/100 [==============================] - 150s 1s/step - loss: 5.2736e-05 - accuracy: 0.3546
66 of 536 332.87032548983893 39.42327845745215
Epoch 1/2
100/100 [==============================] - 131s 1s/step - loss: 5.2177e-05 - accuracy: 0.3509
Epoch 2/2
100/100 [==============================] - 126s 1s/step - loss: 5.2176e-05 - accuracy: 0.3509
67 of 536 337.1798611680667 39.25377488872799
Epoch 1/2
100/100 [==============================] - 139s 1s/step - loss: 5.0386e-05 - accuracy: 0.3508
Epoch 2/2
100/100 [==============================] - 142s 1s/step - loss: 5.0386e-05 - accuracy: 0.3508
68 of 536 341.8827112277349 39.13216327

In [ ]:

lstm_autoencoder2.model.save("LSTM_FullDays")

In [ ]:
output_X = lstm_autoencoder2.model.predict(seq_train)

In [ ]:
reconstruction = np.squeeze(output_X)

In [ ]:
scores = np.linalg.norm(seq_train - reconstruction, axis=-1)

In [ ]:
output_X[1]

In [ ]:
maxes = np.max(scores, axis = 0)

In [ ]:
np.shape(maxes)

In [ ]:
lstm_autoencoder2.plot(maxes, seq_train, threshold=0.95)

In [ ]:
LoopFiles = 3
loops = int(len(BeforeTamping) / LoopFiles) 
if len(files)%LoopFiles != 0:
    loops += 1

st = ti()

Moves = []

for k in range(loops):
    if k == loops -1:
        tfiles = BeforeTamping[k*LoopFiles:]
    else:
        tfiles = BeforeTamping[k*LoopFiles:(k+1)*LoopFiles]
    #Results = Parallel(n_jobs=LoopFiles)(delayed(DeviationVelocity)(file) for file in tfiles)
    Results = Parallel(n_jobs=LoopFiles)(delayed(SmoothMoves)(file) for file in tfiles)
    #Results =[]
    #for file in tfiles:
    #    Results.append(DeviationVelocity(file))
    #    print(file, (ti()-st)/60.0)
    for result in Results:
        Moves.append(result)
    print(k, (ti()-st)/60.0)
        
Sequences = []
Outputs = []
for move in Moves:
    Seq, Out = split_sequences(move.T,TimeSteps,StepSize)
    Sequences.append(Seq)
    Outputs.append(Out)
MoveSegments = []
for seq in Sequences:
    for mv in seq:
        MoveSegments.append(mv)
NextDataPoint = []
for out in Outputs:
    for pt in out:
        NextDataPoint.append(np.reshape(pt,(1,3)))